In [17]:
import sys
import importlib
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from google.colab import files

In [2]:
import sys
sys.path.append('/content')

import myNN
import importlib; importlib.reload(myNN)

print(dir(myNN))  # checking stuff


['ZScoreLayer', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'h5py', 'intList', 'layerNum', 'loadWeights', 'load_model', 'model', 'os', 'tf']


In [3]:
import h5py

# Open the h5 file to inspect its structure
with h5py.File('/content/myNN/weights.h5', 'r') as f:
    # List all groups in the file
    print("Keys in weights.h5:", list(f.keys()))

    # Optionally, explore the first layer or a specific layer
    layer_name = list(f.keys())[0]  # Adjust if needed
    print(f"Structure of layer {layer_name}:")
    print(f[layer_name].attrs)


Keys in weights.h5: ['fc_1_', 'fc_2_', 'fc_3_', 'input_']
Structure of layer fc_1_:
<Attributes of HDF5 object at 137565777820496>


In [4]:
with h5py.File('/content/myNN/weights.h5', 'r') as f:
    # List all groups (layers) in the file
    for group_name in f:
        group = f[group_name]
        print(f"Layer: {group_name}")
        print("    Number of variables:", group.attrs.get('NumVars'))
        print("    Weights:", list(group.keys()))


Layer: fc_1_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: fc_2_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: fc_3_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: input_
    Number of variables: [2.]
    Weights: ['mean', 'stdev']


In [5]:
model = myNN.load_model()

print("Mean shape:", model.get_layer("input_").mean.shape)
print("Mean (first 5):", model.get_layer("input_").mean.numpy()[:5])

model.export('/content/my_saved_NN')

Mean shape: (1024,)
Mean (first 5): [ 71.05658  75.53707  79.59717  87.02772 107.62675]


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_unnormalized']
Received: inputs=Tensor(shape=(None, 1024))
  warnings.warn(msg)


Saved artifact at '/content/my_saved_NN'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1024), dtype=tf.float32, name='input_unnormalized')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  137565778234064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137565778234256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137565778235984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137565778236560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137565778238480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137565778236752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137565778236176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137565778239056: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [18]:
!zip -r my_saved_NN.zip my_saved_NN

files.download("my_saved_NN.zip")

  adding: my_saved_NN/ (stored 0%)
  adding: my_saved_NN/saved_model.pb (deflated 84%)
  adding: my_saved_NN/variables/ (stored 0%)
  adding: my_saved_NN/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: my_saved_NN/variables/variables.index (deflated 59%)
  adding: my_saved_NN/assets/ (stored 0%)
  adding: my_saved_NN/fingerprint.pb (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# Print mean and std from ZScoreLayer
zlayer = None
for layer in model.layers:
    if isinstance(layer, myNN.ZScoreLayer):
        zlayer = layer
        break

if zlayer:
    print("Found ZScoreLayer")
    print("Mean:", zlayer.mean.numpy())
    print("Std:", zlayer.std.numpy())
else:
    print("ZScoreLayer not found in model.")

Found ZScoreLayer
Mean: [ 71.05658   75.53707   79.59717  ...  95.35049  105.896065  84.649895]
Std: [24.894312 27.084814 26.780912 ... 35.477444 41.457195 31.845758]


In [7]:
# === Load and Preprocess Test Data ===
test_df = pd.read_csv('/content/test_7_3label.csv')  # ← Update path if needed

# Separate features and labels
X_test = test_df[[f'var{i}' for i in range(1, 1025)]].values
y_test = test_df['three_label'].values

# Encode labels to integers (e.g., 'correct' → 1, 'incorrect' → 0, 'not_sitting' → 2)
label_map = {'correct': 0, 'incorrect': 1, 'not_sitting': 2}
y_test_encoded = np.array([label_map[label] for label in y_test])

# === Run Inference ===
y_pred_probs = model.predict({'input_unnormalized': X_test})
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # Assumes model output is softmax

# === Evaluate ===
print("Accuracy:", accuracy_score(y_test_encoded, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_encoded, y_pred_classes))
print("Classification Report:\n", classification_report(
    y_test_encoded,
    y_pred_classes,
    target_names=['incorrect', 'correct', 'not_sitting']
))


258/258 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.8959825221507465
Confusion Matrix:
 [[1625  299    0]
 [ 558 5338    0]
 [   0    0  419]]
Classification Report:
               precision    recall  f1-score   support

   incorrect       0.74      0.84      0.79      1924
     correct       0.95      0.91      0.93      5896
 not_sitting       1.00      1.00      1.00       419

    accuracy                           0.90      8239
   macro avg       0.90      0.92      0.91      8239
weighted avg       0.90      0.90      0.90      8239



In [8]:
import tensorflow as tf

# Path to your SavedModel directory
saved_model_dir = "/content/my_saved_NN"

# Converter
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# (Optional) Enable optimizations like quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save it to a file
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


In [15]:
# ================== TESTING TFLITE MODEL ======================

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])


Input shape: [   1 1024]
Output shape: [1 3]


In [16]:
correct = 0
total = len(X_test)

for i in range(total):
    input_data = X_test[i:i+1].astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    pred = np.argmax(output)

    if pred == y_test_encoded[i]:
        correct += 1

print("TFLite accuracy:", correct / total)


TFLite accuracy: 0.8952542784318486
